# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2158958342107,
 'week52high': 146.07,
 'week52low': 80.53,
 'week52highSplitAdjustOnly': 145.8,
 'week52lowSplitAdjustOnly': 79.33,
 'week52change': 0.672012187206659,
 'sharesOutstanding': 17328551979,
 'float': 0,
 'avg10Volume': 97693370,
 'avg30Volume': 97795895,
 'day200MovingAvg': 127.7,
 'day50MovingAvg': 132.73,
 'employees': 148884,
 'ttmEPS': 4.46,
 'ttmDividendRate': 0.8358054452603481,
 'dividendYield': 0.006778321688259369,
 'nextDividendDate': '0',
 'exDividendDate': '2021-04-22',
 'nextEarningsDate': '2021-07-17',
 'peRatio': 28.29317778279686,
 'beta': 1.4007935551215747,
 'maxChangePercent': 49.04078898724934,
 'year5ChangePercent': 5.25245191198159,
 'year2ChangePercent': 1.8162413888685047,
 'year1ChangePercent': 0.6592756308931907,
 'ytdChangePercent': -0.03745932084961823,
 'month6ChangePercent': 0.07384518512177177,
 'month3ChangePercent': -0.0575341997432535,
 'month1ChangePercent': -0.03332649786043605,
 'day30ChangePer

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.6592756308931907

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,135.31,0.622137,N/A
1,AAL,22.50,1.51222,N/A
2,AAP,204.16,0.695443,N/A
3,AAPL,133.21,0.679627,N/A
4,ABBV,117.16,0.3657,N/A
...,...,...,...,...
500,YUM,124.36,0.475266,N/A
501,ZBH,171.81,0.517018,N/A
502,ZBRA,510.46,1.2309,N/A
503,ZION,61.50,1.27504,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,69.27,5.65122,N/A
1,GPS,35.90,4.20897,N/A
2,FCX,43.95,4.17165,N/A
3,IVZ,28.06,3.21447,N/A
4,TPR,48.63,2.78453,N/A
5,KSS,63.85,2.68893,N/A
6,MOS,37.06,2.61371,N/A
7,SIVB,578.79,2.44407,N/A
8,PWR,101.45,2.18366,N/A
9,DFS,120.44,2.16761,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:10000000
10000000


In [9]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

C:\Users\abhin\anaconda\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,69.27,5.65122,2830
1,GPS,35.90,4.20897,5461
2,FCX,43.95,4.17165,4461
3,IVZ,28.06,3.21447,6987
4,TPR,48.63,2.78453,4032
5,KSS,63.85,2.68893,3070
6,MOS,37.06,2.61371,5290
7,SIVB,578.79,2.44407,338
8,PWR,101.45,2.18366,1932
9,DFS,120.44,2.16761,1628


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [13]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.replace(to_replace=[None], value=0, inplace=True)        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,137.41,N/A,0.632732,N/A,0.199723,N/A,0.027637,N/A,-0.006273,N/A,N/A
1,AAL,23.20,N/A,1.490437,N/A,0.840623,N/A,0.306834,N/A,0.001349,N/A,N/A
2,AAP,204.05,N/A,0.689082,N/A,0.324219,N/A,0.330979,N/A,0.083446,N/A,N/A
3,AAPL,129.89,N/A,0.664335,N/A,0.075380,N/A,-0.058936,N/A,-0.034424,N/A,N/A
4,ABBV,120.53,N/A,0.361433,N/A,0.207099,N/A,0.132087,N/A,0.102793,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,122.00,N/A,0.479187,N/A,0.178597,N/A,0.136558,N/A,0.009401,N/A,N/A
501,ZBH,168.42,N/A,0.508913,N/A,0.113027,N/A,0.038187,N/A,-0.006465,N/A,N/A
502,ZBRA,500.60,N/A,1.240072,N/A,0.416568,N/A,0.019346,N/A,-0.028397,N/A,N/A
503,ZION,60.50,N/A,1.302316,N/A,0.644279,N/A,0.240526,N/A,0.076917,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [14]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0      0.522772
1      0.912871
2      0.564356
3      0.546535
4      0.293069
         ...   
500    0.394059
501    0.417822
502    0.841584
503    0.865347
504    0.310891
Name: One-Year Return Percentile, Length: 505, dtype: object
0      0.473267
1      0.968317
2      0.645545
3      0.225743
4      0.493069
         ...   
500    0.437624
501     0.30297
502    0.758416
503    0.912871
504    0.162376
Name: Six-Month Return Percentile, Length: 505, dtype: object
0       0.229703
1       0.922772
2       0.936634
3      0.0990099
4       0.510891
         ...    
500     0.524752
501     0.255446
502          0.2
503     0.813861
504     0.249505
Name: Three-Month Return Percentile, Length: 505, dtype: object
0      0.245545
1      0.283168
2      0.738614
3      0.140594
4      0.845545
         ...   
500    0.324752
501    0.243564
502    0.176238
503    0.710891
504    0.623762
Name: One-Month Return Percentile, Length: 505, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,137.41,N/A,0.632732,0.522772,0.199723,0.473267,0.027637,0.229703,-0.006273,0.245545,N/A
1,AAL,23.20,N/A,1.490437,0.912871,0.840623,0.968317,0.306834,0.922772,0.001349,0.283168,N/A
2,AAP,204.05,N/A,0.689082,0.564356,0.324219,0.645545,0.330979,0.936634,0.083446,0.738614,N/A
3,AAPL,129.89,N/A,0.664335,0.546535,0.075380,0.225743,-0.058936,0.0990099,-0.034424,0.140594,N/A
4,ABBV,120.53,N/A,0.361433,0.293069,0.207099,0.493069,0.132087,0.510891,0.102793,0.845545,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,122.00,N/A,0.479187,0.394059,0.178597,0.437624,0.136558,0.524752,0.009401,0.324752,N/A
501,ZBH,168.42,N/A,0.508913,0.417822,0.113027,0.30297,0.038187,0.255446,-0.006465,0.243564,N/A
502,ZBRA,500.60,N/A,1.240072,0.841584,0.416568,0.758416,0.019346,0.2,-0.028397,0.176238,N/A
503,ZION,60.50,N/A,1.302316,0.865347,0.644279,0.912871,0.240526,0.813861,0.076917,0.710891,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [15]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [16]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:51]

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [17]:
portfolio_input()

Enter the value of your portfolio:10000000


In [18]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

C:\Users\abhin\anaconda\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,137.41,1426,0.632732,0.522772,0.199723,0.473267,0.027637,0.229703,-0.006273,0.245545,0.367822
1,AAL,23.20,8451,1.490437,0.912871,0.840623,0.968317,0.306834,0.922772,0.001349,0.283168,0.771782
2,AAP,204.05,960,0.689082,0.564356,0.324219,0.645545,0.330979,0.936634,0.083446,0.738614,0.721287
3,AAPL,129.89,1509,0.664335,0.546535,0.075380,0.225743,-0.058936,0.0990099,-0.034424,0.140594,0.25297
4,ABBV,120.53,1626,0.361433,0.293069,0.207099,0.493069,0.132087,0.510891,0.102793,0.845545,0.535644
5,ABC,121.49,1613,0.425232,0.342574,0.087053,0.245545,0.115097,0.473267,-0.009053,0.239604,0.325248
6,ABMD,269.27,728,0.476042,0.388119,-0.013421,0.0811881,-0.189204,0.0316832,-0.195330,0.0039604,0.126238
7,ABT,120.81,1623,0.322259,0.263366,0.060210,0.19802,-0.075100,0.0891089,-0.026302,0.182178,0.183168
8,ACN,293.00,669,0.635058,0.524752,0.206965,0.489109,0.124403,0.491089,0.013435,0.338614,0.460891
9,ADBE,494.10,396,0.381923,0.316832,0.038085,0.158416,-0.025537,0.132673,-0.047861,0.10099,0.177228


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [19]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [20]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [21]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [22]:
writer.save()